### N Gram Analysis According To Fourgram

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 1000  # 28 native word end index      

In [4]:
ngram_data_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Data/1-N Gram Data Prepare"

ngram_used_data_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Data/N Gram Used"


path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Result/2-N Gram Analysis According To Fourgram"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def strip(df,column_list):
    '''
    strip(df,column_list) df is dataframe and column_list are selected columns \n
    strip(df_twogram_used,df_twogram_used.columns)
    '''
    for i in column_list:
        new_name = i.strip()
        df.rename(columns={f"{i}":f"{new_name}"}, inplace=True)
        df[f"{new_name}"] = df[f"{new_name}"].apply(lambda x: x.strip())
    
    return df

In [6]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [7]:
def word_in_wordgroup(source_word_list, df_target, target_column, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=False)]  # na=True failed  
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [8]:
def word_in_wordgroup_simple(df, source_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=False)]  # na=True failed    
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

#### Ngram Read

In [9]:
df_twogram_used = pd.read_excel(f"{ngram_used_data_path}/{lang_folder.capitalize()}_Twogram_Used.xlsx")
df_twogram_used = strip(df_twogram_used,df_twogram_used.columns)
df_twogram_used.drop_duplicates(inplace=True)
df_twogram_used.reset_index(drop=True, inplace=True)
df_twogram_used

,twogram
0,bir şey
1,bu kadar
2,ne kadar
3,ve de
4,ne için
...,...
152,senin gibi
153,sorun değil
154,sorun yok
155,tamam mı


In [10]:
df_threegram_used = pd.read_excel(f"{ngram_used_data_path}/{lang_folder.capitalize()}_Threegram_Used.xlsx")
df_threegram_used = strip(df_threegram_used, df_threegram_used.columns)
df_threegram_used.drop_duplicates(inplace=True)
df_threegram_used.reset_index(drop=True, inplace=True)
df_threegram_used

,threegram
0,ama neden böyle
1,ama o benim
2,ama şimdi olmaz
3,artık herkes biliyor
4,aynı şekilde devam
...,...
333,bir daha hiç
334,bir de şu
335,bir daha asla
336,peki ne için


In [11]:
twogram_used_list = list(set(df_twogram_used["twogram"]))
threegram_used_list = list(set(df_threegram_used["threegram"]))

In [12]:
df_twogram = pd.read_csv(f"{ngram_data_path}/{lang_folder.capitalize()}_{word_end}_Word_Twogram_Selected.csv")
df_twogram_remove_used = df_twogram[~df_twogram["twogram"].isin(twogram_used_list)]
df_twogram_remove_used.reset_index(drop=True, inplace=True)
df_twogram_remove_used

,twogram,frequency
0,hayır hayır,266457
1,özür dilerim,260661
2,ne yapıyorsun,236176
3,hiçbir şey,209701
4,ne demek,188846
...,...,...
220006,dışarıda bizimle,6
220007,bizim kimseye,6
220008,tarafa bunu,6
220009,beri kahrolası,6


In [13]:
df_threegram = pd.read_csv(f"{ngram_data_path}/{lang_folder.capitalize()}_{word_end}_Word_Threegram_Selected.csv")
df_threegram_remove_used = df_threegram[~df_threegram["threegram"].isin(threegram_used_list)]
df_threegram_remove_used.reset_index(drop=True, inplace=True)
df_threegram_remove_used

,threegram,frequency
0,her şey yolunda,75968
1,hayır hayır hayır,69443
2,ne demek istiyorsun,63142
3,sana bir şey,56713
4,böyle bir şey,51970
...,...,...
500006,sonra insanlar bu,10
500007,o anda dünya,10
500008,zaman zaman sizinle,10
500009,bir güç sahibi,10


In [14]:
df_fourgram = pd.read_csv(f"{ngram_data_path}/{lang_folder.capitalize()}_{word_end}_Word_Fourgram_Selected.csv")
df_fourgram

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
366580,sadece yemek ve içki,7
366581,küçük bir şey ve,7
366582,olduğunu bu yüzden biliyorum,7
366583,sadece yeri değil zamanı,7


In [15]:
df_fourgram_selected_for_course = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment3/\
Data/Manuel Select/Selected 4grams.xlsx")
df_fourgram_selected_for_course

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
95,her şey iyi olacak,2479
96,öyle bir şey değil,2478
97,bunu sen mi yaptın,2453
98,ne demek oluyor bu,2431


In [16]:
twogram_list = df_twogram_remove_used["twogram"].to_list()  # without used
threegram_list = df_threegram_remove_used["threegram"].to_list()  # without used
#fourgram_list = df_fourgram["fourgram"].to_list()
fourgram_course_list = df_fourgram_selected_for_course["fourgram"].to_list()

#### Given Previous Course

##### Twogram Used In Threegram Used In Fourgram (1A)

In [17]:
# parameter
source_wordgroup_list = threegram_used_list
search_column_name = "threegram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [18]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_threegram_used_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_threegram_used_in_selected_four

,threegram,fourgram
0,şey var mı,bir şey var mı
1,bu kadar uzun,neden bu kadar uzun
2,daha önce de,bunu daha önce de
3,var biliyor musun,ne var biliyor musun
4,neden bu kadar,neden bu kadar uzun
5,neden bu kadar,neden bu kadar çok
6,ben de seni,ben de seni seviyorum
7,evet ben de,evet ben de öyle
8,ol tamam mı,sakin ol tamam mı
9,sen de biliyorsun,bunu sen de biliyorsun


In [19]:
df_threegram_used_in_selected_four2 = df_threegram_used_in_selected_four[["threegram"]]
df_threegram_used_in_selected_four2.drop_duplicates(inplace=True)
df_threegram_used_in_selected_four2.reset_index(drop=True, inplace=True)
df_threegram_used_in_selected_four2

<ipython-input-19-c0b3da00885b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_threegram_used_in_selected_four2.drop_duplicates(inplace=True)


,threegram
0,şey var mı
1,bu kadar uzun
2,daha önce de
3,var biliyor musun
4,neden bu kadar
5,ben de seni
6,evet ben de
7,ol tamam mı
8,sen de biliyorsun
9,bu kadar mı


In [20]:
# parameter
source_wordgroup_list = twogram_used_list
search_column_name = "twogram"
target_column = "threegram"
df_target = df_threegram_used_in_selected_four2

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [21]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_twogram_used_in_threegram_used_in_selected_four2 = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_twogram_used_in_threegram_used_in_selected_four2

,twogram,threegram
0,bu da,bu da ne
1,uzun zaman,çok uzun zaman
2,uzun zaman,uzun zaman oldu
3,çok güzel,çok güzel değil
4,daha önce,daha önce de
5,bu kadar,bu kadar uzun
6,evet ben,evet ben de
7,teşekkür ederim,çok teşekkür ederim
8,daha önce,daha önce hiç
9,bu kadar,neden bu kadar


In [22]:
df_twogram_used_in_three_used_in_selected_four = pd.merge(df_twogram_used_in_threegram_used_in_selected_four2, df_threegram_used_in_selected_four, how="inner", on="threegram")
df_twogram_used_in_three_used_in_selected_four.drop_duplicates(inplace=True)
df_twogram_used_in_three_used_in_selected_four

,twogram,threegram,fourgram
0,bu da,bu da ne,bu da ne demek
1,bu da,bu da ne,bu da ne böyle
2,uzun zaman,çok uzun zaman,çok uzun zaman oldu
3,uzun zaman,uzun zaman oldu,çok uzun zaman oldu
4,çok güzel,çok güzel değil,çok güzel değil mi
5,daha önce,daha önce de,bunu daha önce de
6,bu kadar,bu kadar uzun,neden bu kadar uzun
7,evet ben,evet ben de,evet ben de öyle
8,ben de,evet ben de,evet ben de öyle
9,teşekkür ederim,çok teşekkür ederim,için çok teşekkür ederim


In [23]:
df_twogram_used_in_three_used_in_selected_four.to_excel(f"{lang_folder.capitalize()}_Twogram_Used_In_Threegram_Used_In_Selected_Fourgram_(1A)_Result.xlsx", index=False)  # 1A

##### Twogram Used In Selected Fourgram (2A)

In [24]:
# parameter
source_wordgroup_list = twogram_used_list
search_column_name = "twogram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [25]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_used_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_used_in_selected_four

,twogram,fourgram
0,bu da,bu da ne demek
1,tamam mı,sakin ol tamam mı
2,bu da,bu da ne böyle
3,bu da,bu da neydi böyle
4,en son,en son ne zaman
...,...,...
81,bir şey,kötü bir şey mi
82,bir şey,bir şey söyleyeyim mi
83,bir şey,öyle bir şey olmayacak
84,evet mi,evet mi hayır mı


In [26]:
df_two_used_in_selected_four2 = df_two_used_in_selected_four[~df_two_used_in_selected_four["fourgram"].isin(list(set(df_twogram_used_in_three_used_in_selected_four["fourgram"])))]
df_two_used_in_selected_four2.reset_index(drop=True, inplace=True)
df_two_used_in_selected_four2

,twogram,fourgram
0,bu da,bu da neydi böyle
1,en son,en son ne zaman
2,ne var,ne var ne yok
3,ya da,öyle ya da böyle
4,daha var,bir tane daha var
5,onun gibi,onun gibi bir şey
6,sen mi,bunu sen mi yaptın
7,çok güzel,seni görmek çok güzel
8,senin için,senin için ne yapabilirim
9,her şeyi,için her şeyi yaparım


In [27]:
df_two_used_in_selected_four2["fourgram"].nunique()

34

In [28]:
df_two_used_in_selected_four2.to_excel(f"{lang_folder.capitalize()}_Twogram_Used_In_Selected_Fourgram_(2A)_Result.xlsx", index=False)  # 2A

##### Threegram Used In Selected Fourgram (4A)

In [29]:
# parameter
source_wordgroup_list = threegram_used_list
search_column_name = "threegram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [30]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_three_used_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_three_used_in_selected_four

,threegram,fourgram
0,şey var mı,bir şey var mı
1,ben de seni,ben de seni seviyorum
2,daha önce de,bunu daha önce de
3,bu kadar uzun,neden bu kadar uzun
4,evet ben de,evet ben de öyle
5,sen de biliyorsun,bunu sen de biliyorsun
6,neden bu kadar,neden bu kadar uzun
7,neden bu kadar,neden bu kadar çok
8,bir şey daha,bir şey daha var
9,bir şey daha,başka bir şey daha


In [31]:
df_three_used_in_selected_four2 = df_three_used_in_selected_four[~df_three_used_in_selected_four["fourgram"].isin(list(set(df_twogram_used_in_three_used_in_selected_four["fourgram"])))]
df_three_used_in_selected_four2.reset_index(drop=True, inplace=True)
df_three_used_in_selected_four2

,threegram,fourgram


In [32]:
df_three_used_in_selected_four2.to_excel(f"{lang_folder.capitalize()}_Threegram_Used_In_Selected_Fourgram_(4A)_Result.xlsx", index=False)  # 4A

##### Twogram Used In Threegram Used (3A)

In [33]:
set_two_1A_list = list(set(df_twogram_used_in_three_used_in_selected_four["twogram"]))
set_two_2A_list = list(set(df_two_used_in_selected_four2["twogram"]))
set_two_used_list = set_two_1A_list + set_two_2A_list
#set_two_used_list

In [34]:
df_two_used_remove_used = df_twogram_used[~df_twogram_used["twogram"].isin(set_two_used_list)]
df_two_used_remove_used.reset_index(drop=True, inplace=True)
df_two_used_remove_used

,twogram
0,ve de
1,ne için
2,o kadar
3,bu çok
4,ama değil
...,...
123,öyle mi
124,para için
125,senin gibi
126,sorun değil


In [35]:
set_three_1A_list = list(set(df_twogram_used_in_three_used_in_selected_four["threegram"]))
set_three_4A_list = list(set(df_three_used_in_selected_four2["threegram"]))
set_three_used_list = set_three_1A_list + set_three_4A_list
#set_three_used_list

In [36]:
df_three_used_remove_used = df_threegram_used[~df_threegram_used["threegram"].isin(set_three_used_list)]
df_three_used_remove_used.reset_index(drop=True, inplace=True)
df_three_used_remove_used

,threegram
0,ama neden böyle
1,ama o benim
2,ama şimdi olmaz
3,artık herkes biliyor
4,aynı şekilde devam
...,...
303,bir daha hiç
304,bir de şu
305,bir daha asla
306,peki ne için


In [37]:
df_two_used_remove_used_list = list(set(df_two_used_remove_used["twogram"]))

In [38]:
# parameter
source_wordgroup_list = df_two_used_remove_used_list
search_column_name = "twogram"
target_column = "threegram"
df_target = df_three_used_remove_used

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [39]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_used_in_three_used_not_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_used_in_three_used_not_in_selected_four

,twogram,threegram
0,evet o,evet o benim
1,evet o,evet o kadar
2,aynı şeyi,aynı şeyi ben
3,aynı şeyi,aynı şeyi mi
4,o da,o da çok
...,...,...
264,evet öyle,evet öyle değil
265,tabii ki,tabii ki öyle
266,evet öyle,evet öyle oldu
267,tabii ki,tabii ki var


In [40]:
df_two_used_in_three_used_not_in_selected_four.to_excel(f"{lang_folder.capitalize()}_Twogram_Used_In_Threegram_Used_Not_In_Selected_Fourgram_(3A)_Result.xlsx", index=False)  # 3A 

##### Section 5

In [41]:
#set_two_used_all = set_two_used_list + list(set(df_two_used_in_three_used_not_in_selected_four["twogram"]))
#set_two_all

In [42]:
#df_twogram_used_alone = df_twogram_used[~df_twogram_used["twogram"].isin(set_two_used_all)]
#df_twogram_used_alone.reset_index(drop=True, inplace=True)
#df_twogram_used_alone

In [43]:
#df_twogram_used_alone.to_excel(f"{lang_folder.capitalize()}_Twogram_Used_Alone_(5A)_Result.xlsx", index=False)  # 5A

##### Section 6

In [44]:
#set_three_used_all = set_three_used_list + list(set(df_two_used_in_three_used_not_in_selected_four["threegram"]))

In [45]:
#df_threegram_used_alone = df_threegram_used[~df_threegram_used["threegram"].isin(set_three_used_all)]
#df_threegram_used_alone.reset_index(drop=True, inplace=True)
#df_threegram_used_alone

In [46]:
#df_threegram_used_alone.to_excel(f"{lang_folder.capitalize()}_Threegram_Used_Alone_(6A)_Result.xlsx", index=False)

##### Section 7

In [47]:
#set_four_used_all = list(set(df_three_used_in_selected_four2["fourgram"])) + list(set(df_two_used_in_selected_four2["fourgram"])) + list(set(df_twogram_used_in_three_used_in_selected_four["fourgram"]))

In [48]:
#df_fourgram_used_alone = df_fourgram_selected_for_course[~df_fourgram_selected_for_course["fourgram"].isin(set_four_used_all)]
#df_fourgram_used_alone.reset_index(drop=True, inplace=True)
#df_fourgram_used_alone

In [49]:
#df_fourgram_used_alone.to_excel(f"{lang_folder.capitalize()}_Fourgram_Used_Alone_(7A)_Result.xlsx", index=False)  # 7A

#### Give Reveal Course

##### Twogram In Threegram In Selected Fourgram (1B)

In [50]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [51]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_threegram_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_threegram_in_selected_four

,threegram,fourgram
0,her şey yolunda,her şey yolunda mı
1,sana bir şey,sana bir şey söyleyeyim
2,böyle bir şey,hiç böyle bir şey
3,böyle bir şey,böyle bir şey olmayacak
4,ne işin var,burada ne işin var
...,...,...
142,sakin ol tamam,sakin ol tamam mı
143,istiyor musun istemiyor,istiyor musun istemiyor musun
144,şey söyleyeyim mi,bir şey söyleyeyim mi
145,olur ne olmaz,ne olur ne olmaz


In [52]:
df_threegram_in_selected_four2 = df_threegram_in_selected_four[["threegram"]]
df_threegram_in_selected_four2.drop_duplicates(inplace=True)
df_threegram_in_selected_four2.reset_index(drop=True, inplace=True)
df_threegram_in_selected_four2

<ipython-input-52-b6cd2c5833f9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_threegram_in_selected_four2.drop_duplicates(inplace=True)


,threegram
0,her şey yolunda
1,sana bir şey
2,böyle bir şey
3,ne işin var
4,öyle bir şey
...,...
128,sakin ol tamam
129,istiyor musun istemiyor
130,şey söyleyeyim mi
131,olur ne olmaz


In [53]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "threegram"
df_target = df_threegram_in_selected_four2

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [54]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_twogram_in_threegram_in_selected_four2 = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_twogram_in_threegram_in_selected_four2

,twogram,threegram
0,ne demek,da ne demek
1,ne demek,ne demek oluyor
2,ne demek,ne demek biliyor
3,ne zaman,son ne zaman
4,da ne,da ne demek
...,...,...
206,şey buldun,bir şey buldun
207,şey buldun,şey buldun mu
208,öyle ya,öyle ya da
209,şey kontrol,her şey kontrol


In [55]:
df_twogram_in_three_in_selected_four = pd.merge(df_twogram_in_threegram_in_selected_four2, df_threegram_in_selected_four, how="inner", on="threegram")
df_twogram_in_three_in_selected_four.drop_duplicates(inplace=True)
df_twogram_in_three_in_selected_four

,twogram,threegram,fourgram
0,ne demek,da ne demek,bu da ne demek
1,ne demek,da ne demek,da ne demek oluyor
2,da ne,da ne demek,bu da ne demek
3,da ne,da ne demek,da ne demek oluyor
4,ne demek,ne demek oluyor,da ne demek oluyor
...,...,...,...
227,mi hayır,evet mi hayır,evet mi hayır mı
228,yapıyorsun değil,yapıyorsun değil mi,şaka yapıyorsun değil mi
229,şey buldun,bir şey buldun,bir şey buldun mu
230,öyle ya,öyle ya da,öyle ya da böyle


In [57]:
df_twogram_in_three_in_selected_four["fourgram"].nunique()

90

In [56]:
df_twogram_in_three_in_selected_four.to_excel(f"{lang_folder.capitalize()}_Twogram_In_Threegram_In_Selected_Fourgram_(1B)_Result.xlsx", index=False)  # 1B

##### Twogram In Selected Fourgram (2B)

In [58]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [59]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_in_selected_four

,twogram,fourgram
0,ne demek,bu da ne demek
1,ne demek,da ne demek oluyor
2,ne demek,bu ne demek oluyor
3,ne demek,ne demek biliyor musun
4,ne demek,ne demek oluyor bu
...,...,...
209,mi hayır,evet mi hayır mı
210,yapıyorsun değil,şaka yapıyorsun değil mi
211,şey buldun,bir şey buldun mu
212,öyle ya,öyle ya da böyle


In [60]:
#df_two_in_selected_four_list = df_two_in_selected_four.groupby(["twogram"])["fourgram"].apply(list).reset_index(name="fourgram")
#df_two_in_selected_four_list

In [61]:
df_two_in_selected_four2 = df_two_in_selected_four[~df_two_in_selected_four["fourgram"].isin(list(set(df_twogram_in_three_in_selected_four["fourgram"])))]
df_two_in_selected_four2.reset_index(drop=True, inplace=True)
df_two_in_selected_four2

,twogram,fourgram
0,başka bir,başka bir şey var
1,başka bir,başka bir şey yok
2,başka bir,başka bir şey değil
3,başka bir,başka bir şey daha
4,şey yok,başka bir şey yok
5,şey var,bir şey var mı
6,şey var,başka bir şey var
7,de öyle,evet ben de öyle
8,şey değil,başka bir şey değil
9,neden bu,neden bu kadar uzun


In [62]:
df_two_in_selected_four2["fourgram"].nunique()

10

In [63]:
df_two_in_selected_four2.to_excel(f"{lang_folder.capitalize()}_Twogram_In_Selected_Fourgram_(2B)_Result.xlsx", index=False)  # 2B

##### Threegram In Selected Fourgram (4B)

In [64]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [65]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_three_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_three_in_selected_four

,threegram,fourgram
0,her şey yolunda,her şey yolunda mı
1,sana bir şey,sana bir şey söyleyeyim
2,böyle bir şey,hiç böyle bir şey
3,böyle bir şey,böyle bir şey olmayacak
4,ne işin var,burada ne işin var
...,...,...
142,sakin ol tamam,sakin ol tamam mı
143,istiyor musun istemiyor,istiyor musun istemiyor musun
144,şey söyleyeyim mi,bir şey söyleyeyim mi
145,olur ne olmaz,ne olur ne olmaz


In [66]:
#df_three_in_selected_four_list = df_three_in_selected_four.groupby(["threegram"])["fourgram"].apply(list).reset_index(name="fourgram")
#df_three_in_selected_four_list

In [67]:
df_three_in_selected_four2 = df_three_in_selected_four[~df_three_in_selected_four["fourgram"].isin(list(set(df_twogram_in_three_in_selected_four["fourgram"])))]
df_three_in_selected_four2.reset_index(drop=True, inplace=True)
df_three_in_selected_four2

,threegram,fourgram


In [68]:
df_three_in_selected_four2.to_excel(f"{lang_folder.capitalize()}_Threegram_In_Selected_Fourgram_(4B)_Result.xlsx", index=False)  # 4B

##### Twogram In Threegram (3B)

In [69]:
set_two_1B_list = list(set(df_twogram_in_three_in_selected_four["twogram"]))
set_two_2B_list = list(set(df_two_in_selected_four2["twogram"]))
set_two_list = set_two_1B_list + set_two_2B_list
#set_two_list

In [70]:
df_two_remove_used = df_twogram_remove_used[~df_twogram_remove_used["twogram"].isin(set_two_list)]
df_two_remove_used

,twogram,frequency
0,hayır hayır,266457
1,özür dilerim,260661
2,ne yapıyorsun,236176
3,hiçbir şey,209701
5,bu bir,146347
...,...,...
220006,dışarıda bizimle,6
220007,bizim kimseye,6
220008,tarafa bunu,6
220009,beri kahrolası,6


In [71]:
set_three_1B_list = list(set(df_twogram_in_three_in_selected_four["threegram"]))
set_three_4B_list = list(set(df_three_in_selected_four2["threegram"]))
set_three_list = set_three_1B_list+set_three_4B_list
#set_three_list

In [72]:
df_three_remove_used = df_threegram_remove_used[~df_threegram_remove_used["threegram"].isin(set_three_list)]
df_three_remove_used

,threegram,frequency
1,hayır hayır hayır,69443
2,ne demek istiyorsun,63142
5,sen iyi misin,42180
7,için özür dilerim,36255
8,daha iyi bir,35136
...,...,...
500006,sonra insanlar bu,10
500007,o anda dünya,10
500008,zaman zaman sizinle,10
500009,bir güç sahibi,10


In [73]:
df_two_remove_used_list = list(set(df_two_remove_used["twogram"]))

In [74]:
# parameter
source_wordgroup_list = df_two_remove_used_list
search_column_name = "twogram"
target_column = "threegram"
df_target = df_three_remove_used

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [75]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_three_not_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_in_three_not_in_selected_four

,twogram,threegram
0,teşekkürler gerçekten,teşekkürler gerçekten çok
1,teşekkürler gerçekten,için teşekkürler gerçekten
2,teşekkürler gerçekten,çok teşekkürler gerçekten
3,yapacaksın doktor,ne yapacaksın doktor
4,sevgili sevgili,benim sevgili sevgili
...,...,...
929252,elbette çok,elbette çok güzel
929253,elbette çok,elbette çok iyi
929254,elbette çok,elbette çok kötü
929255,elbette çok,elbette çok büyük


In [76]:
df_two_in_three_not_in_selected_four_freq = pd.merge(df_two_in_three_not_in_selected_four, df_threegram, how="left", on="threegram")
df_two_in_three_not_in_selected_four_freq.drop_duplicates(inplace=True)
df_two_in_three_not_in_selected_four_freq.sort_values(by="frequency", ascending=False, inplace=True)
df_two_in_three_not_in_selected_four_freq_select = df_two_in_three_not_in_selected_four_freq.head(100)
df_two_in_three_not_in_selected_four_freq_select

,twogram,threegram,frequency
442512,hayır hayır,hayır hayır hayır,69443
869849,demek istiyorsun,ne demek istiyorsun,63142
113604,sen iyi,sen iyi misin,42180
894885,iyi misin,sen iyi misin,42180
786927,özür dilerim,için özür dilerim,36255
...,...,...,...
570468,de onu,ben de onu,10742
167324,harika değil,harika değil mi,10734
705966,için iyi,için iyi bir,10733
503737,de mi,sen de mi,10685


In [77]:
df_two_in_three_not_in_selected_four_freq_select.to_excel(f"{lang_folder.capitalize()}_Twogram_In_Threegram_Not_In_Selected_Fourgram_(3B)_Result.xlsx", index=False)

##### Section 5

In [78]:
set_two_all = set_two_list + list(set(df_two_in_three_not_in_selected_four["twogram"]))
#set_two_all

In [79]:
df_twogram_alone = df_twogram_remove_used[~df_twogram_remove_used["twogram"].isin(set_two_all)]
df_twogram_alone.reset_index(drop=True, inplace=True)
df_twogram_alone

,twogram,frequency
0,nasılsın dostum,2516
1,merhaba nasılsın,2149
2,hayır değildi,2027
3,neden acaba,1617
4,selam kızlar,1476
...,...,...
84816,dışarıda bizimle,6
84817,bizim kimseye,6
84818,tarafa bunu,6
84819,beri kahrolası,6


In [80]:
df_twogram_alone_select = df_twogram_alone.head(100)
df_twogram_alone_select

,twogram,frequency
0,nasılsın dostum,2516
1,merhaba nasılsın,2149
2,hayır değildi,2027
3,neden acaba,1617
4,selam kızlar,1476
...,...,...
95,hayır yoktu,547
96,hey merhaba,545
97,evet anlaşıldı,542
98,durun bekleyin,540


In [81]:
df_twogram_alone_select.to_excel(f"{lang_folder.capitalize()}_Twogram_Alone_(5)_Result.xlsx", index=False)  # 5

##### Section 6

In [82]:
set_three_all = set_three_list + list(set(df_two_in_three_not_in_selected_four["threegram"]))

In [83]:
df_threegram_alone = df_threegram_remove_used[~df_threegram_remove_used["threegram"].isin(set_three_all)]
df_threegram_alone.reset_index(drop=True, inplace=True)
df_threegram_alone

,threegram,frequency
0,daha iyi bir,35136
1,o kadar da,33638
2,çok güzel bir,30839
3,çok iyi bir,29751
4,ne kadar da,18799
...,...,...
1689,o zaman dön,10
1690,seninle tanrının arasında,10
1691,şey yolunda dedim,10
1692,gitsin öyle mi,10


In [84]:
df_threegram_alone_select = df_threegram_alone.head(100)
df_threegram_alone_select

,threegram,frequency
0,daha iyi bir,35136
1,o kadar da,33638
2,çok güzel bir,30839
3,çok iyi bir,29751
4,ne kadar da,18799
...,...,...
95,başka bir şeyin,1084
96,kötü bir şeyler,1057
97,için çok uzun,1056
98,bu ne güzel,1031


In [85]:
df_threegram_alone_select.to_excel(f"{lang_folder.capitalize()}_Threegram_Alone_(6)_Result.xlsx", index=False)  # 6

##### Section 7

In [86]:
set_four_all = list(set(df_three_in_selected_four2["fourgram"])) + list(set(df_two_in_selected_four2["fourgram"])) + list(set(df_twogram_in_three_in_selected_four["fourgram"]))

In [87]:
df_fourgram_alone = df_fourgram_selected_for_course[~df_fourgram_selected_for_course["fourgram"].isin(set_four_all)]
df_fourgram_alone.reset_index(drop=True, inplace=True)
df_fourgram_alone

,fourgram,frequency


In [88]:
df_fourgram_alone_select = df_fourgram_alone.head(100)
df_fourgram_alone_select

,fourgram,frequency


In [89]:
df_fourgram_alone_select.to_excel(f"{lang_folder.capitalize()}_Fourgram_Alone_(7)_Result.xlsx", index=False) 

#### Copy Move And Delete

In [90]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_Result*")
#output_file = glob.glob(f"*_Selected_*.xlsx")
output_file

['Turkish_Twogram_Alone_(5)_Result.xlsx',
 'Turkish_Twogram_Used_In_Threegram_Used_In_Selected_Fourgram_(1A)_Result.xlsx',
 'Turkish_Twogram_Used_In_Selected_Fourgram_(2A)_Result.xlsx',
 'Turkish_Threegram_Used_In_Selected_Fourgram_(4A)_Result.xlsx',
 'Turkish_Twogram_Used_In_Threegram_Used_Not_In_Selected_Fourgram_(3A)_Result.xlsx',
 'Turkish_Twogram_In_Threegram_In_Selected_Fourgram_(1B)_Result.xlsx',
 'Turkish_Twogram_In_Selected_Fourgram_(2B)_Result.xlsx',
 'Turkish_Threegram_In_Selected_Fourgram_(4B)_Result.xlsx',
 'Turkish_Twogram_In_Threegram_Not_In_Selected_Fourgram_(3B)_Result.xlsx',
 'Turkish_Threegram_Alone_(6)_Result.xlsx',
 'Turkish_Fourgram_Alone_(7)_Result.xlsx']

In [91]:
for l in output_file:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [92]:
for j in output_file:
    try:
        os.remove(j)
    except:
        pass